In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib inline
from pathlib import Path
import functools
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib import rcParams 

from matplottoy.data import fruit
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [5]:
data = fruit.FruitSection(df)

In [6]:
df.index.inferred_type

'integer'

In [7]:
data.local_section({'calories':(17, 20)})

FruitSection(section=   fruit  calories  juice
2  lemon        17  False
3   lime        20  False)

In [8]:
data.projection('fruit').values()

array(['apple', 'orange', 'lemon', 'lime'], dtype=object)

In [9]:
# should K be up front or last?
def constant_color(c):
     return lambda x: [mcolors.to_rgba(c) for _ in x]

def constant_value(val):
     return lambda x: [val for _ in x]

In [10]:
constant_color('k')([1,2,3,4])
constant_value(1)([1,2,3,4])

[1, 1, 1, 1]

In [83]:
# multivariate nu, make key always a tuple ([possibly data class])
V = {'height': bar.Mu(lambda x : x, ('calories',)), 
     'position': bar.Mu(position.Nominal({'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}), 'fruit'),
     'floor': bar.Mu(constant_value(0), None), 
     'width': bar.Mu(constant_value(.8), None),
     'facecolor': bar.Mu(color.Nominal({True: 'orange', False: 'blue'}), ('juice', 'fruit',)), 
     'edgecolor':bar.Mu(constant_color('k'), None),
     'linewidth':bar.Mu(constant_value(2), None),
     'linestyle':bar.Mu(constant_value((0, None)),  None)}

In [84]:
BB = bar.Bar()

In [85]:
BB.V

{'linestyle': Mu(nu=None, F=None),
 'height': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'width': Mu(nu=None, F=None)}

In [86]:
BN = BB.compose_with_nu(**V)

In [87]:
BN.V

{'linestyle': Mu(nu=<function constant_value.<locals>.<lambda> at 0x000001C103BC8DC0>, F=None),
 'height': Mu(nu=<function <lambda> at 0x000001C103BC8F70>, F='calories'),
 'position': Mu(nu=<matplottoy.encoders.position.Nominal object at 0x000001C103BD61C0>, F='fruit'),
 'floor': Mu(nu=<function constant_value.<locals>.<lambda> at 0x000001C103BC8C10>, F=None),
 'linewidth': Mu(nu=<function constant_value.<locals>.<lambda> at 0x000001C103BC8310>, F=None),
 'facecolor': Mu(nu=<matplottoy.encoders.color.Nominal object at 0x000001C103BD6280>, F='juice'),
 'edgecolor': Mu(nu=<function constant_color.<locals>.<lambda> at 0x000001C103BC8B80>, F=None),
 'width': Mu(nu=<function constant_value.<locals>.<lambda> at 0x000001C103BC8430>, F=None)}

In [88]:
BB.V

{'linestyle': Mu(nu=None, F=None),
 'height': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'width': Mu(nu=None, F=None)}

In [89]:
BN = BN.compose_with_nu(
    edgecolor=bar.Mu(F=None, nu=constant_color('darkgray')))

In [90]:
BN.compose_with_nu(Pi=, nu=, *fields)

Mu(nu=None, F=None)

In [91]:
BN.V['edgecolor']

Mu(nu=<function constant_color.<locals>.<lambda> at 0x000001C103BC8E50>, F=None)

In [92]:
BN.make_mu(data.local_section())

{'orientation': 'v',
 'linestyle': [(0, None), (0, None), (0, None), (0, None)],
 'height': array([95, 67, 17, 20], dtype=int64),
 'position': array([0, 2, 4, 6]),
 'floor': [0, 0, 0, 0],
 'linewidth': [2, 2, 2, 2],
 'facecolor': [RGBA(r=1.0, g=0.6470588235294118, b=0.0, a=1.0),
  RGBA(r=1.0, g=0.6470588235294118, b=0.0, a=1.0),
  RGBA(r=0.0, g=0.0, b=1.0, a=1.0),
  RGBA(r=0.0, g=0.0, b=1.0, a=1.0)],
 'edgecolor': [(0.6627450980392157,
   0.6627450980392157,
   0.6627450980392157,
   1.0),
  (0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0),
  (0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0),
  (0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0)],
 'width': [0.8, 0.8, 0.8, 0.8]}

In [93]:
for (k, v) in BN.V.items():
    print(k, v.F, data.local_section().projection(v.F).values())
    v.nu(data.local_section().projection(v.F).values())

linestyle None [nan nan nan nan]
height calories [95 67 17 20]
position fruit ['apple' 'orange' 'lemon' 'lime']
floor None [nan nan nan nan]
linewidth None [nan nan nan nan]
facecolor juice [ True  True False False]
edgecolor None [nan nan nan nan]
width None [nan nan nan nan]


In [94]:
bar.QHatData(BN)

In [95]:
bwd = bar.QHatData(BN).compose_with_tau(data)

In [96]:
bwd.query()

FruitSection(section=    fruit  calories  juice
0   apple        95   True
1  orange        67   True
2   lemon        17  False
3    lime        20  False)

In [97]:
A = bar.GenericArtist(bwd)

In [98]:
dir(bwd.graphic)

['P',
 'V',
 '_V',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'compose_with_nu',
 'make_mu',
 'orientation',
 'qhat']

In [99]:
A.topArt

In [100]:
f = lambda x : np.zeros(len(x))
f(bwd.section.projection(None).values())

array([0., 0., 0., 0.])

In [102]:
from matplotlib.backends.backend_agg import RendererAgg
renderer = RendererAgg(500, 400, 100)
A.draw(renderer)

Path(array([[ 0. ,  0. ],
       [ 0. , 95. ],
       [ 0.8, 95. ],
       [ 0.8,  0. ],
       [ 0. ,  0. ]]), None)


SystemError: new style getargs format but argument is not a tuple